<a href="https://colab.research.google.com/github/ajcries/Onepiece/blob/main/SGA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Subtitle Generator App
# This app generates subtitle files only (no burning)
# Updated with WhisperX for better alignment, and optional VTT output for CC

# 1. INSTALL DEPENDENCIES
!pip install -q whisperx deep-translator fonttools moviepy gradio --no-deps

import os
import datetime
import zipfile
import subprocess
import shutil
import re

import whisperx
import moviepy.editor as mp
import gradio as gr
from deep_translator import GoogleTranslator
from fontTools.ttLib import TTFont

# ─── UTILITY FUNCTIONS ───────────────────────────────────────────────────────

def format_timestamp_srt(seconds: float) -> str:
    td = datetime.timedelta(seconds=seconds)
    total = int(td.total_seconds())
    h = total // 3600
    m = (total % 3600) // 60
    s = total % 60
    ms = int(td.microseconds / 1000)
    return f"{h:02d}:{m:02d}:{s:02d},{ms:03d}"


def format_timestamp_ass(seconds: float) -> str:
    td = datetime.timedelta(seconds=seconds)
    total = int(td.total_seconds())
    h = total // 3600
    m = (total % 3600) // 60
    s = total % 60
    cs = int(td.microseconds / 10000)
    return f"{h}:{m:02d}:{s:02d}.{cs:02d}"


def format_timestamp_vtt(seconds: float) -> str:
    td = datetime.timedelta(seconds=seconds)
    total = int(td.total_seconds())
    h = total // 3600
    m = (total % 3600) // 60
    s = total % 60
    ms = int(td.microseconds / 1000)
    return f"{h:02d}:{m:02d}:{s:02d}.{ms:03d}"


def hex_to_ass_color(hex_str: str) -> str:
    h = hex_str.lstrip('#')
    return f"&H00{h[4:6]}{h[2:4]}{h[0:2]}&"


def get_font_internal_name(font_path: str) -> str:
    try:
        font = TTFont(font_path)
        for record in font['name'].names:
            if record.nameID == 4:  # Full font name
                return record.toUnicode()
        for record in font['name'].names:
            if record.nameID == 1:  # Family name
                return record.toUnicode()
    except:
        pass
    return os.path.basename(font_path).rsplit('.', 1)[0]


def install_font(font_file_obj):
    if not font_file_obj:
        return "Arial"

    temp_font_dir = "/tmp/custom_fonts"
    os.makedirs(temp_font_dir, exist_ok=True)

    fname = os.path.basename(font_file_obj.name)
    temp_path = os.path.join(temp_font_dir, fname)

    shutil.copy(font_file_obj.name, temp_path)

    system_dir = "/usr/share/fonts/truetype/custom"
    os.makedirs(system_dir, exist_ok=True)
    shutil.copy(temp_path, os.path.join(system_dir, fname))

    subprocess.run(["fc-cache", "-fv"], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)

    return get_font_internal_name(temp_path)


# ─── SUBTITLE PARSING HELPERS ───────────────────────────────────────────────

def parse_srt_to_ass_lines(srt_path: str) -> list:
    lines = []
    try:
        with open(srt_path, 'r', encoding='utf-8') as f:
            blocks = f.read().split('\n\n')
        for block in blocks:
            if not block.strip(): continue
            parts = block.strip().split('\n', 2)
            if len(parts) < 3: continue
            time_line = parts[1]
            if '-->' not in time_line: continue
            start_raw, end_raw = time_line.split('-->')
            start = format_timestamp_ass(float(start_raw.replace(',', '.')))
            end   = format_timestamp_ass(float(end_raw.replace(',', '.')))
            text = parts[2].replace('\n', '\\N').strip()
            lines.append(f"Dialogue: 0,{start},{end},Default,,0,0,0,,{text}\n")
    except:
        pass
    return lines


def parse_ass_dialogue_lines(ass_path: str) -> list:
    lines = []
    try:
        with open(ass_path, 'r', encoding='utf-8') as f:
            for line in f:
                if line.startswith("Dialogue:"):
                    lines.append(line.rstrip('\n') + '\n')
    except:
        pass
    return lines


# ─── MAIN PROCESSING FUNCTION ───────────────────────────────────────────────

def process_subs(
    videos,
    subtitles,
    target_lang,
    font_size,
    text_color,
    outline_color,
    custom_font_file,
    vertical_position,
    generate_vtt,
    progress = gr.Progress()
):
    if not videos:
        return None, "No files uploaded."

    final_font = install_font(custom_font_file)

    output_sub_files = []
    log_lines = []

    ass_color_primary = hex_to_ass_color(text_color)
    ass_color_outline  = hex_to_ass_color(outline_color)

    ass_header = f"""[Script Info]
ScriptType: v4.00+
PlayResX: 640
PlayResY: 360
WrapStyle: 0
ScaledBorderAndShadow: yes

[V4+ Styles]
Format: Name, Fontname, Fontsize, PrimaryColour, SecondaryColour, OutlineColour, BackColour, Bold, Italic, Underline, StrikeOut, ScaleX, ScaleY, Spacing, Angle, BorderStyle, Outline, Shadow, Alignment, MarginL, MarginR, MarginV, Encoding
Style: Default,{final_font},{font_size},{ass_color_primary},&H000000FF&,{ass_color_outline},&H00000000&,0,0,0,0,100,100,0,0,1,2,2,2,{vertical_position},10,10,1

[Events]
Format: Layer, Start, End, Style, Name, MarginL, MarginR, MarginV, Effect, Text
"""

    # Build map of base name → subtitle path
    sub_map = {}
    if subtitles:
        for f in subtitles:
            base = os.path.basename(f.name).rsplit('.', 1)[0]
            sub_map[base] = f.name

    device = "cuda"
    compute_type = "float16"
    model = whisperx.load_model("large-v3", device=device, compute_type=compute_type)

    for vid_idx, video in enumerate(progress.tqdm(videos, desc="Processing media")):
        input_path = video.name
        base_name = os.path.basename(input_path).rsplit('.', 1)[0]
        log_lines.append(f"→ {base_name}")

        is_video_file = input_path.lower().endswith(('.mp4','.mkv','.mov','.avi','.webm'))

        # ── Get subtitle content ─────────────────────────────────────────────
        ass_dialogue_lines = []

        if base_name in sub_map:
            sub_path = sub_map[base_name]
            log_lines.append("   Using uploaded subtitle")
            if sub_path.lower().endswith('.srt'):
                ass_dialogue_lines = parse_srt_to_ass_lines(sub_path)
            elif sub_path.lower().endswith('.ass'):
                ass_dialogue_lines = parse_ass_dialogue_lines(sub_path)
        else:
            log_lines.append("   AI transcribing + translating...")
            temp_wav = f"temp_{base_name}.wav"
            try:
                clip = mp.AudioFileClip(input_path) if not is_video_file else mp.VideoFileClip(input_path)
                clip.audio.write_audiofile(temp_wav, logger=None)
                audio = whisperx.load_audio(temp_wav)
                task = "translate" if target_lang.lower() == "english" else "transcribe"
                result = model.transcribe(audio, language="ja", task=task)

                # Align
                align_lang = result["language"]
                align_model, metadata = whisperx.load_align_model(language_code=align_lang, device=device)
                result_aligned = whisperx.align(result["segments"], align_model, metadata, audio, device, return_char_alignments=False)

                segments = result_aligned["segments"]

                for seg in segments:
                    text = seg["text"].strip()
                    if target_lang.lower() not in ["english", "japanese"]:
                        try:
                            text = GoogleTranslator(source='auto', target=target_lang).translate(text)
                        except:
                            pass
                    start = format_timestamp_ass(seg["start"])
                    end   = format_timestamp_ass(seg["end"])
                    ass_dialogue_lines.append(
                        f"Dialogue: 0,{start},{end},Default,,0,0,0,,{text.replace('\n','\\N')}\n"
                    )
            except Exception as e:
                log_lines.append(f"   ERROR during transcription: {str(e)}")
            finally:
                if os.path.exists(temp_wav):
                    os.remove(temp_wav)

        if not ass_dialogue_lines:
            log_lines.append("   No subtitle content generated or parsed → skipping")
            continue

        # ── Save .ass ────────────────────────────────────────────────────────
        ass_path = f"{base_name}_styled.ass"
        with open(ass_path, "w", encoding="utf-8") as f:
            f.write(ass_header + "".join(ass_dialogue_lines))
        output_sub_files.append(ass_path)

        # ── Save .srt (clean version) ───────────────────────────────────
        srt_lines = []
        for i, dlg in enumerate(ass_dialogue_lines):
            parts = dlg.split(',', 3)
            if len(parts) < 4: continue
            start_ass = parts[1]
            end_ass   = parts[2]
            text = parts[3].strip()
            start_srt = start_ass.replace('.', ',')[:-2] + "0"
            end_srt   = end_ass.replace('.', ',')[:-2] + "0"
            srt_lines.append(f"{i+1}\n{start_srt} --> {end_srt}\n{text.replace('\\N','\n')}\n\n")

        srt_path = f"{base_name}_{target_lang}.srt"
        with open(srt_path, "w", encoding="utf-8") as f:
            f.writelines(srt_lines)
        output_sub_files.append(srt_path)

        # ── Optional .vtt for CC ─────────────────────────────────────────
        if generate_vtt:
            vtt_lines = ["WEBVTT\n\n"]
            i = 0
            while i < len(srt_lines):
                if srt_lines[i].strip().isdigit():
                    i += 1
                    continue
                if "-->" in srt_lines[i]:
                    vtt_lines.append(srt_lines[i].replace(",", "."))
                    i += 1
                text = ""
                while i < len(srt_lines) and srt_lines[i].strip() and "-->" not in srt_lines[i]:
                    text += srt_lines[i] + "\n"
                    i += 1
                if text:
                    vtt_lines.append(text.strip() + "\n\n")
                i += 1
            vtt_path = f"{base_name}_{target_lang}.vtt"
            with open(vtt_path, "w", encoding="utf-8") as f:
                f.writelines(vtt_lines)
            output_sub_files.append(vtt_path)

        log_lines.append("")

    # ── ZIP RESULTS ──────────────────────────────────────────────────────────
    sub_zip_path = None
    if output_sub_files:
        sub_zip_path = "subtitles.zip"
        with zipfile.ZipFile(sub_zip_path, "w", zipfile.ZIP_DEFLATED) as zf:
            for p in output_sub_files:
                zf.write(p)
                os.remove(p)

    log_text = "\n".join(log_lines) + f"\n\nDone. Processed {len(videos)} file(s)."
    return sub_zip_path, log_text


# ─── GRADIO INTERFACE ────────────────────────────────────────────────────────

with gr.Blocks(theme=gr.themes.Soft()) as demo:
    gr.Markdown("# 🎌 Subtitle Generator")
    gr.Markdown("• AI transcribe Japanese video/audio with improved alignment • Restyle existing .srt/.ass • Custom font from upload")

    with gr.Row():
        with gr.Column(scale=6):
            videos_input = gr.File(
                label="Video / Audio files (batch supported)",
                file_count="multiple",
                file_types=["video", "audio"]
            )
            subs_input = gr.File(
                label="Optional: existing subtitles (.srt or .ass) — name must match base name",
                file_count="multiple",
                file_types=[".srt", ".ass"]
            )

            target_language = gr.Dropdown(
                choices=["english", "japanese", "spanish", "french", "german", "korean", "chinese (simplified)", "russian"],
                value="english",
                label="Target language (used only when no subtitle is provided)"
            )

            vtt_box = gr.Checkbox(False, label="Generate VTT format for closed captions (in addition to SRT/ASS)")

            with gr.Accordion("Subtitle Appearance", open=True):
                font_upload = gr.File(
                    label="Custom font (.ttf / .otf) — uploaded each session",
                    file_types=[".ttf", ".otf"]
                )
                font_size_slider = gr.Slider(12, 60, 26, step=1, label="Font size")
                vert_pos_slider = gr.Slider(10, 340, 30, label="Vertical position (higher = higher on screen)")
                text_col = gr.ColorPicker("#FFFFFF", label="Text color")
                border_col = gr.ColorPicker("#000000", label="Outline color")

            process_btn = gr.Button("Generate Subtitles", variant="primary", size="lg")

        with gr.Column(scale=4):
            subs_zip = gr.File(label="Download subtitles (.srt + .ass + optional .vtt)")
            processing_log = gr.Textbox(label="Processing log", lines=16, interactive=False)

    process_btn.click(
        process_subs,
        inputs=[
            videos_input,
            subs_input,
            target_language,
            font_size_slider,
            text_col,
            border_col,
            font_upload,
            vert_pos_slider,
            vtt_box
        ],
        outputs=[subs_zip, processing_log]
    )

demo.launch(share=True, debug=False)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 106.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 4.8 MB/s eta 0:00:00


/usr/local/lib/python3.12/dist-packages/moviepy/config_defaults.py:47: SyntaxWarning: invalid escape sequence '\P'
  IMAGEMAGICK_BINARY = r"C:\Program Files\ImageMagick-6.8.8-Q16\magick.exe"
/usr/local/lib/python3.12/dist-packages/moviepy/video/io/ffmpeg_reader.py:294: SyntaxWarning: invalid escape sequence '\d'
  lines_video = [l for l in lines if ' Video: ' in l and re.search('\d+x\d+', l)]
/usr/local/lib/python3.12/dist-packages/moviepy/video/io/ffmpeg_reader.py:367: SyntaxWarning: invalid escape sequence '\d'
  rotation_lines = [l for l in lines if 'rotate          :' in l and re.search('\d+$', l)]
/usr/local/lib/python3.12/dist-packages/moviepy/video/io/ffmpeg_reader.py:370: SyntaxWarning: invalid escape sequence '\d'
  match = re.search('\d+$', rotation_line)
  if event.key is 'enter':

  with gr.Blocks(theme=gr.themes.Soft()) as demo:



Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://fa35a3da7182c8e842.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
